In [117]:
import numpy as np
from pobs.utils import get_param_from_json, save_json, load_json, load_pickle, load_hdf5, save_hdf5
import matplotlib.pyplot as plt
import pobs

In [108]:
import pobs
from pobs.pobs import POBS
import pobs.utils as utils
import pobs.mp as mp
import pobs.modelgenerator as modelgenerator

from importlib import reload
reload(pobs)
reload(pobs.utils)
reload(pobs.modelgenerator)
reload(pobs.mp)
reload(pobs.pobs)

<module 'pobs.pobs' from '/Users/phurailatpamhemantakumar/phd/mypackages/pobs/pobs/pobs.py'>

In [140]:
test = pobs.POBS(
    posterior1='/Users/phurailatpamhemantakumar/phd/mypackages/pobs/all_data/pe_results/unlensed/label_index_1_mc_53_q_0p9_time_1249637317_data0_1249637317-2160294_analysis_H1L1V1_result.hdf5',
    posterior2='/Users/phurailatpamhemantakumar/phd/mypackages/pobs/all_data/pe_results/unlensed/label_index_2_mc_33_q_0p6_time_1240188813_data0_1240188813-6522114_analysis_H1L1V1_result.hdf5',
    # create_new=True,
    kde_model_type="gaussian_kde",
    spin_zero=True,
    npool=1,
)

# test = pobs.POBS(
#     posterior1=None,
#     posterior2=None,
#     create_new=True,
#     spin_zero=True,
#     kde_model_type="jax_gaussian_kde",
# )

astro_lensed is None
getting default astro_lensed data_dict from pobs module
creating a new astro_lensed model...
astro_unlensed is None
getting default astro_unlensed data_dict from pobs module
creating a new astro_unlensed1 model...
creating a new astro_unlensed2 model...
pe_prior is None
getting default pe_prior data_dict from pobs module
creating a new pe_prior model...
creating a new posterior1 model...
creating a new posterior2 model...
creating a new posterior_combine model...


In [116]:
# npool=1, size=500: 13.4s
# npool=2, size=500: 26.8s
# npool=8, size=500: 51.s
# for loop iter=1, 14.7s
# for loop iter=8, 17.5s
bf = test.bayes_factor_multiprocessing(sample_size=500)

In [155]:
bf = test.bayes_factor(sample_size=2000)

In [142]:
from multiprocessing import Pool
import pobs_mp
reload(pobs_mp)

<module 'pobs_mp' from '/Users/phurailatpamhemantakumar/phd/mypackages/pobs/dev/pobs_mp.py'>

In [153]:
# 25.6s 4 processes
npool = 4
input_arguments = [[50, ] for _ in range(npool)]
results = []
with Pool(processes=npool) as pool:
    for result in pool.map(pobs_mp.pobs_mp, input_arguments):
        results.append(list(result))

# # with for loop
# for input_argument in input_arguments:
#     results.append(pobs_mp.pobs_mp(input_argument))

astro_lensed is Noneastro_lensed is Noneastro_lensed is Noneastro_lensed is None



getting default astro_lensed data_dict from pobs modulegetting default astro_lensed data_dict from pobs modulegetting default astro_lensed data_dict from pobs modulegetting default astro_lensed data_dict from pobs module



creating a new astro_lensed model...
astro_unlensed is Nonecreating a new astro_lensed model...
creating a new astro_lensed model...creating a new astro_lensed model...

getting default astro_unlensed data_dict from pobs module

astro_unlensed is Noneastro_unlensed is Noneastro_unlensed is None


getting default astro_unlensed data_dict from pobs modulegetting default astro_unlensed data_dict from pobs modulegetting default astro_unlensed data_dict from pobs module


creating a new astro_unlensed1 model...
creating a new astro_unlensed2 model...
pe_prior is None
getting default pe_prior data_dict from pobs module
creating a new pe_prior model...
creating a new astro_unlensed1 model..

In [154]:
results

[[2.686296483115308e-20, -19.570846056494148],
 [2.686296483115308e-20, -19.570846056494148],
 [2.686296483115308e-20, -19.570846056494148],
 [2.686296483115308e-20, -19.570846056494148]]

In [133]:
import numpy as np
from pobs.utils import (
    load_json,
    load_pickle,
)


def get_pdf_fuctions(model_name = 'astro_lensed', pobs_directory="./pobs_data"):

    sample_size_original = input_arguments[0]  
    sample_size = sample_size_original 

    kde = load_pickle(f"{pobs_directory}/model_path_{model_name}.pkl")
    scaler = load_pickle(f"{pobs_directory}/scaler_path_{model_name}.pkl")
    min_max = load_json(f"{pobs_directory}/min_max_path_{model_name}.pkl")

    return kde, scaler, min_max

def resample(size=10000, model_path=None, scaler_path=None, min_max_path=None, batch_size=None):

    if model_path is None:
        model_path = self.model_path
    if scaler_path is None:
        scaler_path = self.scaler_path
    if min_max_path is None:
        min_max_path = self.min_max_path
    if label_list is None:
        label_list = self.path_dict['label_list']
    if batch_size is None:
        batch_size = self.batch_size
        if batch_size > size:
            batch_size = size
        

    # get the model, scaler and min_max
    kde, scaler, min_max = self.get_model_scaler_minmax(model_path, scaler_path, min_max_path)

    result_dict = {}
    for i in range(len(label_list)):
        result_dict[label_list[i]] = np.array([])

    while True:

        # generate random data
        # kde.set_bandwidth(bw_method=kde.factor * self.bandwidth_factor)
        if self.model_type == "gaussian_kde":
            data = kde.resample(batch_size).T
        elif self.model_type == "jax_gaussian_kde":
            key = random.PRNGKey(seed=np.random.randint(0, 1000000))
            data = kde.resample(key, (batch_size,)).T
            data = np.array(data)

        # inverse transform the data
        data = scaler.inverse_transform(data)

        j = 0
        idx = np.ones(data.shape[0], dtype=bool)
        for key, value in min_max.items():
            min_data = value['min_data']
            max_data = value['max_data']
            data_ = data[:, j]
            idx &= (data_<min_data) | (data_>max_data)
        
        for i in range(len(label_list)):
            result_dict[label_list[i]] = np.concatenate([result_dict[label_list[i]], data[:, i][~idx]])

        final_size = result_dict[label_list[0]].shape[0]
        batch_size = (size - final_size) + 1
        if final_size >= size:
            for key, value in result_dict.items():
                result_dict[key] = value[:size]
            break

    return result_dict

In [134]:
astro_lensed = get_pdf_fuctions()

In [ ]:
astro_lensed

In [127]:
input_arguments = [10]
astro_lensed_kde_ = bayes_factor_multiprocessing(input_arguments)
astro_lensed_kde_.resample(10)

array([[-0.61503638, -0.74866135,  0.24152157, -1.44291363, -1.33853124,
         1.47327916, -1.57709265,  1.19461686,  1.24707814,  0.74584194],
       [ 0.10040368, -0.3745733 ,  0.3267378 , -1.56910991, -0.96340884,
         0.85243165, -1.42487465,  0.1293556 ,  1.5205656 ,  0.77866181],
       [-0.31384805, -0.72872967,  1.00996818,  1.21904015, -0.99859245,
         1.60245971, -0.57532634, -0.76385511,  0.76484034,  0.85692834],
       [ 1.59527263,  1.64104488, -1.91212678,  1.75121268,  2.60774335,
         0.44304811, -0.9659416 , -0.57801501,  1.06413758,  1.14607394],
       [ 0.73530195, -0.4550562 , -0.77174412, -0.8638894 ,  1.68626801,
        -0.80272827, -1.76674764, -1.18009203,  1.19732631,  0.9298448 ],
       [ 0.78229582,  0.28480183,  0.49014744, -2.36967677, -0.20982676,
        -0.27754086, -2.53044786,  0.92147707,  0.41904365,  1.27092107],
       [ 0.77549517, -1.64878099,  1.05636498, -2.06376538, -0.22696937,
         0.36604402, -2.15075522, -0.06521441

In [97]:
# with multiprocessing, 4 cores: 40.8/4 + overhead ~ 11s
# bf = test.po_hemanta_numerator(sample_size=500)

In [85]:
bf

(8.754331644107801e-14, -13.057777004846763)

In [99]:
bf = test.bayes_factor(sample_size=500)

In [100]:
bf

(1.3278202411787063e-11, -10.876860715290531)

In [19]:
# non-jax: size: 10000, time=40.08s, bf = (3.813393853299081e-15, -14.418688337794492)
# jax: size: 10000, bf = (7.523307222052605e-17, -16.12359120295313)

# non-jax: size=100000, 6m46s, bf = (2.4315613155861754e-14, -13.61411477454944)
# jax: size: 100000, bf = (1.3828888913558608e-13, -12.859212712016124)
bf

[100, 100, 100, 100]

In [ ]:
import get_model
import load_model
# import path dict all

def bayes_factor_mp(input_arguments):

    size = input_arguments[0]
    path_dict = input_arguments[1]
    # load the path dict all json file
    # get all model, scalar, min_max
    # load model
    # find pdf

